# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

from smdebug import modes
from smdebug.profiler.utils import str2bool
from smdebug.pytorch import get_hook

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
[2023-02-13 21:00:12.963 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:18 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


ModuleNotFoundError: No module named 'torch'

In [4]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tuner import (IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner)
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
import boto3

role= sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [5]:
#TODO: Fetch and upload the data to AWS S3
inputs = "s3://sagemaker-us-east-1-535830523552/dogimages/"
print(inputs)
# Command to download and unzip data
#!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip dogImages.zip


s3://sagemaker-us-east-1-535830523552/dogimages/


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [6]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges= {"lr": ContinuousParameter(0.001, 0.1),
                        "epochs":IntegerParameter(2, 4),
                        "batch-size":CategoricalParameter([32, 64, 128, 256, 512])}

objective_metric_name= "average test loss"
objective_type= "Minimize"
metric_definition=[{"Name":"average test loss", "Regex":"Test set: Average loss:([0-9\\.]+)"}]

In [7]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point = "hpo.py",
    role= role,
    py_version="py36",
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)# TODO: Your estimator here

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definition,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type= objective_type
)
    # TODO: Your HP tuner here

In [11]:
# TODO: Fit your HP Tuner
tuner.fit({"training":inputs}) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


...........

KeyboardInterrupt: 

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()
#TODO

#Get the hyperparameters of the best trained model
HP_best_trained = best_estimator.hyperparameters()
best_hyperparameters = {"batch_size": int(HP_best_trained['batch_size'].replace('"', '')), \
                   "learning_rate": HP_best_trained['learning_rate']}
best_hyperparameters

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

rules = [ Rule.sagemaker(rule_configs.vanishing_gradient()),
          Rule.sagemaker(rule_configs.overfit()),
          Rule.sagemaker(rule_configs.overtraining()),
          Rule.sagemaker(rule_configs.poor_weight_utilization()),
          Rule.sagemaker(rule_configs.loss_not_decreasing()),
          ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
          ProfilerRule.sagemaker(rule_configs.ProfilerReport()) ]

from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
profiler_config = ProfilerConfig(system_monitor_interval_millis=500, framework_profile_params = FrameworkProfile(num_steps=10))
debugger_config = DebuggerHookConfig(hook_parameters={"train.save_interval":"100", "test.save_interval":"10"})
        


In [ ]:
# TODO: Create and fit an estimator
# TODO: Your estimator here
estimator = PyTorch(role = role,
                    instance_count=1,
                    instance_type= "ml.m5.xlarge",
                    source_dir="",
                    entry_point="train_model.py",
                    hyperparameters = best_hyperparameters,
                    framework_version="1.8",
                    py_version="py36",
                    rules= rules,
                    profiler_config= profiler_config,
                    debugger_hook_config= debugger_config)

estimator.fit(wait=True)


In [ ]:
# TODO: Plot a debugging output.
job_name = estimator.latest_training_job.name
client = estimator.sagemaker_session.sagemaker_client
describtion = client.describe_training_job(TrainingJobName=estimator.latest_training_job.name)

from smdebug.trials import create_trial
from smdebug.core.modes import Modekeys
trial = create_trial(estimator.latest_job_debugger_artifacts_path())
print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

#Set up functions to plot the output tensors
def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals


import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot

def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()
plot_tensor(trial, "CrossEntropyLoss_output_0")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
session = boto3.session.Session()
region = session.region_name
training_job_name = estimator.latest_training_job.name
print(f"training jobname:{training_job_name}")
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)


## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor

model_location=

jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )
        
pytorch_model = PyTorchModel(model_data=model_location, role=role, entry_point='infernce_1.py',py_version='py3',
                             framework_version='1.4',
                             predictor_cls=ImagePredictor)


predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large') # TODO: Add your deployment configuration like instance type and number of instances

In [20]:
# TODO: Run an prediction on the endpoint
from PIL import Image
import io
import requests
#request_dict={ "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" }
request_dict={ "url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg" }

# TODO: Your code to load and preprocess image to send to endpoint for prediction
image = requests.get(request_dict['url']).content
type(image)
Image.open(io.BytesIO(image))

response = predictor.predict(image, initial_args={"ContentType": "image/jpeg"})

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pc\\dogImages\\t10k-images-idx3-ubyte.gz'

In [ ]:
import json
response2=predictor.predict(json.dumps(request_dict), initial_args={"ContentType": "application/json"})
type(response2[0][0])
response2[0]


In [ ]:
import torch
import numpy as np
np.argmax(response, 1)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
#predictor.delete_endpoint()